In [1]:
# Load the normalized tables

import pickle
import sys
sys.setrecursionlimit(10000)

with open("normal_tables1_1.txt", "rb") as fp:
    nt1 = pickle.load(fp)
    
with open("normal_tables1_2.txt", "rb") as fp:
    nt2 = pickle.load(fp)
    
with open("normal_tables2_1.txt", "rb") as fp:
    nt3 = pickle.load(fp)
    
with open("normal_tables2_2.txt", "rb") as fp:
    nt4 = pickle.load(fp)

In [6]:
# # split tables again

import itertools
from collections import defaultdict

def splitDict(d):
    n = len(d) // 2          # length of smaller half
    i = iter(d.items())      # alternatively, i = d.iteritems() works in Python 2

    d1 = dict(itertools.islice(i, n))   # grab first n items
    d2 = dict(i)                        # grab the rest

    return d1, d2

# save the split tables to normalize by running 4 scripts concurrently
import pickle

# https://stackoverflow.com/a/26496899
def default_to_regular(d):
    if isinstance(d, defaultdict):
        d = {k: default_to_regular(v) for k, v in d.items()}
    return d

def splitDictWrite(d, name1, name2):
    d1,d2 = splitDict(d)
    d1 = default_to_regular(d1)
    d2 = default_to_regular(d2)
    
    with open(name1, "wb") as fp:
         pickle.dump(d1, fp)

    with open(name2, "wb") as fp:
         pickle.dump(d2, fp)
            
def loadDict(d):
    with open(d, "rb") as fp:
        n = pickle.load(fp)
    return n

In [7]:
splitDictWrite(nt1,"nt1_1","nt1_2")
splitDictWrite(nt2,"nt2_1","nt2_2")
splitDictWrite(nt3,"nt3_1","nt3_2")
splitDictWrite(nt4,"nt4_1","nt4_2")

nt1_1 = loadDict("nt1_1")
nt1_2 = loadDict("nt1_2")
nt2_1 = loadDict("nt2_1")
nt2_2 = loadDict("nt2_2")
nt3_1 = loadDict("nt3_1")
nt3_2 = loadDict("nt3_2")
nt4_1 = loadDict("nt4_1")
nt4_2 = loadDict("nt4_2")

# len(nt1) + len(nt2) + len(nt3) + len(nt4) == \
len(nt1_1) + len(nt1_2) + len(nt2_1) + len(nt2_2) + len(nt3_1) + len(nt3_2) + len(nt4_1) + len(nt4_2)

186

In [4]:
# number of articles containing tables with disambiguated entities (186)
# len(nt1) + len(nt2) + len(nt3) + len(nt4)

In [5]:
# set up SPARQL endpoint for wikidata
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [6]:
# resolve Wikidata entity from title

from wikitables.client import Client

client = Client("en")

def getWikidata(title):
    return client.fetch_wikidata(title)

def retrieveExtract(article):
    return client.fetch_extract(article)

In [7]:
# Features (as suggested by the authors (Emir Munoz & Aidan Hogan) of Wikitables Triples Paper)
# =============
# Table Features
#     (-) 1 Number of rows
#     (-) 2 Number of columns
#     (-) 3 Total relations extracted
# Column Features
#     (+) 4 Potential relations
#     (+) 5 Unique potential relations
#     (+) 6 Entity relatedness (new)
# Predicate Features
#     (+) 7 Normalized unique subject count / Normalized unique object count
# Cell Features
#     (-) 8 Number of entities in subject cell
#     (-) 9 Number of entities in object cell
#     (-) 10 String length in subject cell
#     (-) 11 String length in object cell
# Predicate/Column Features
#     (+) 12 Maximum between Jaro-Walker distance and dice coefficient
#     (+) 13 Number of rows where the relation holds
#     (+) 14 Number of relations in KB for all possible relations
#     (+) 15 Number of relations in KB for all unique relations
# Where (+) signifies a positive feature & (-) signifies a negative feature

In [8]:
# Predicate Features

# subject and object must be prefixed with "wd:"
# if the object is a value it must be double quoted
#      getPredicates("wd:Q69","'2830'")
# we get subject -> object, and its inverse object -> subject
def getPredicates(subject,obj, number = False):
    if number:
        # we do a different query and return only the non-inverse
        sparql.setQuery("""SELECT * WHERE
        {
             %s ?p %s .
             FILTER(STRSTARTS(str(?p), "http://www.wikidata.org/prop/direct/"))
             SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
        }""" % (subject, obj))
        
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        predicates = list()

        for row in results["results"]["bindings"]:
            if row["p"]["type"] == "uri":
                predicates.append(row["p"]["value"])

        return predicates
    
    sparql.setQuery("""SELECT DISTINCT ?p1 ?p2
    {
         {%s ?p1 %s 
         FILTER(STRSTARTS(str(?p1), "http://www.wikidata.org/prop/direct/"))} 
         UNION {%s ?p2 %s
         FILTER(STRSTARTS(str(?p2), "http://www.wikidata.org/prop/direct/"))}
         SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
    }""" % (subject, obj, obj, subject))
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    predicates_so = list()
    predicates_os = list()
    
    for row in results["results"]["bindings"]:
        try:
            if row["p1"]["type"] == "uri":
                predicates_so.append(row["p1"]["value"])
            if row["p2"]["type"] == "uri":
                predicates_os.append(row["p2"]["value"])
        except:
            pass

    return predicates_so, predicates_os

def checkTriple(subject, predicate, obj):
    sparql.setQuery("""ASK
    {
         %s %s %s .
         SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
    }""" % (subject, predicate, obj))
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    return results["boolean"]
    
# (+) 13,14,15 Normalized unique subject count / Normalized unique object count

In [9]:
# Predicate/ Column Features
# (+) 12 Max of dice coeffient and jaro-winkler distance
from pyjarowinkler import distance

def dice_coefficient(a,b):
    if not len(a) or not len(b): return 0.0
    """ quick case for true duplicates """
    if a == b: return 1.0
    """ if a != b, and a or b are single chars, then they can't possibly match """
    if len(a) == 1 or len(b) == 1: return 0.0
    
    """ use python list comprehension, preferred over list.append() """
    a_bigram_list = [a[i:i+2] for i in range(len(a)-1)]
    b_bigram_list = [b[i:i+2] for i in range(len(b)-1)]
    
    a_bigram_list.sort()
    b_bigram_list.sort()
    
    # assignments to save function calls
    lena = len(a_bigram_list)
    lenb = len(b_bigram_list)
    # initialize match counters
    matches = i = j = 0
    while (i < lena and j < lenb):
        if a_bigram_list[i] == b_bigram_list[j]:
            matches += 2
            i += 1
            j += 1
        elif a_bigram_list[i] < b_bigram_list[j]:
            i += 1
        else:
            j += 1
    
    score = float(matches)/float(lena + lenb)
    return score

def feature11(string1, string2):
    return max(distance.get_jaro_distance(string1, string2, winkler=True, scaling=0.1),\
              dice_coefficient(string1, string2))

# (+) 12 No of rows that contain the subject and object
# def feature12(predicate):
#     sparql.setQuery("""SELECT * WHERE
#     {
#          ?s %s ?o .
#          FILTER(STRSTARTS(str(?s), "http://www.wikidata.org/entity/"))
#          FILTER(STRSTARTS(str(?o), "http://www.wikidata.org/entity/"))
#          SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
#     }""" % (predicate))
    
#     sparql.setReturnFormat(JSON)
#     results = sparql.query().convert()

#     return len(results["results"]["bindings"])

In [10]:
# functions entitiy relatedness using API call
import tagme

with open("tagme", 'r') as file:
    token = file.readline().strip()
    
tagme.GCUBE_TOKEN = token

def disambig(text, min_rho=None):
    annotations = tagme.annotate(text)
    a = dict()
    for x in annotations.annotations:
        if min_rho is None or x.score > min_rho:
            a[str(x.mention)] = x.entity_title
        
    return a

# Get relatedness between a pair of entities specified by title.
# rels = tagme.relatedness_title(("Barack Obama", "Italy"))
# print("Obama and italy have a semantic relation of", rels.relatedness[0].rel)

In [11]:
import simplemediawiki 

wiki = simplemediawiki.MediaWiki('https://www.wikidata.org/w/api.php')

def findProperty(string, amount = None):
    results = wiki.call({'action': 'wbsearchentities', 'search': string, 'type': 'property', 'language': 'en', 'limit': 10})

    properties = list()
    for i,x in enumerate(results['search']):
        if amount and amount == i:
            break
        properties.append(x['id'])
    
    return properties

In [12]:
# iterate through normalized and regular table

import re
def find_number(string):
    return re.findall('\d+', string)

import itertools

def get_pairs(l):
    return list(itertools.combinations(l,2))

def flatten(l):
    return list(itertools.chain.from_iterable(l))

def getCol(value, row):
    for i,x in enumerate(row):
        if isinstance(x, list):
            for i2, v in enumerate(x):
                if value == v: 
                    return (i,i2)
        elif value == x:
            return i
        
    # didn't make it
    print(value, row)

        
def getSOPred(string1, string2, fnt, existingtriples, row = list(), section_entities = list(), pos = None):
    e1 = string1.replace("'",'"')
    e2 = string2.replace("'",'"')
    n1 = False
    n2  = False
    predicates = set()
    if string1 not in fnt and string1 not in section_entities:
        # not an actual entity; double quote
        e1 = "'" + e1 + "'"
        n1 = True
    else:
        # must catch exception because forgot to parse out deadlinks
        try:
            e1 = "wd:" + getWikidata(e1)
        except:
            e1 = "'" + e1 + "'"
            n1 = True
    if string2 not in fnt and string2 not in section_entities:
        # not an actual entity; double quote
        e2 = "'" + e2 + "'"
        n2 = True
    else:
        # must catch exception because forgot to parse out deadlinks
        try:
            e2 = "wd:" + getWikidata(e2)
        except:
            e2 = "'" + e2 + "'"
            n2 = True

    if n1 and n2:
        return

    if not n1 and not n2:
        pred1, pred2 = getPredicates(e1,e2)

        if pred1:
            # e1 ?p e2
            if not pos:
                s = (x, getCol(str(string1),row))
            else:
                s = pos
            o = (x, getCol(str(string2), row))
            for p1 in pred1:
                existing_triples.append((s,p1,o))
                predicates.add(p1)
        if pred2:
            # e2 ?p e1
            if not pos:
                o = (x, getCol(str(string1), row))
            else:
                o = pos
            s = (x, getCol(str(string2), row))
            for p1 in pred2:
                existing_triples.append((s,p1,o))
                predicates.add(p1)

    elif n1 and not n2:
        pred = getPredicates(e2,e1,True)
        if pred:
            # e2 ?p e1
            if not pos:
                o = (x, getCol(str(string1), row))
            else:
                o = pos
            s = (x, getCol(str(string2), row))
            for p1 in pred:
                existing_triples.append((s,p1,o))
                predicates.add(p1)

    elif not n1 and n2:
        pred = getPredicates(e1,e2,True)
        if pred:
            # e1 ?p e2
            if not pos:
                s = (x, getCol(str(string1), row))
            else:
                s = pos
            o = (x, getCol(str(string2), row))
            for p1 in pred:
                existing_triples.append((s,p1,o))
                predicates.add(p1)   
    
    return predicates

In [13]:
# helper functions to genreate candidate triples
def checkPositionValid(pos, table):
    try:
        x = table[pos[0]][pos[1][0]][pos[1][1]]
        return True
    except:
        return False

def generateAllTuples(t, table):
    s = set()
    if t == (None, 'article') or t[1] == 'sub':
        s.add((None,t))
        return s
    for i in range(1,len(table)):
        pos = (i, t)
        if checkPositionValid(pos, table):
            s.add(pos)
    return s

def getElementsPosition(t):
    s = set()
    for x in t:
        s.add(x[1])
    return s

def getValueFromTable(pos, table,k,section_entities):
    if pos == (None, (None, 'article')):
            return k
    elif pos[1][1] == 'sub':
            return section_entities[pos[1][0]]
    
    return table[pos[0]][pos[1][0]][pos[1][1]]
    

# triple candidate generation
def generateCandidates(xextracted,xextractedtable):
    scount = defaultdict(int)
    ocount = defaultdict(int)
    scountset_o = defaultdict(set)
    ocountset_s = defaultdict(set)

    candidate_triples = set()
    candidate_preds = set()


    # count the occurences of subject and object if > threshold then suggest candidate triples & extract features
    for t in set(xextracted):
        scount[(t[0][1], t[1].split('/')[-1])]+=1
        scountset_o[(t[0][1], t[1].split('/')[-1])].add(t[2])
        ocount[(t[1].split('/')[-1], t[2][1])]+=1
        ocountset_s[(t[1].split('/')[-1], t[2][1])].add(t[0])

    PROPERTY_STRING = "http://www.wikidata.org/prop/direct/"

    # do some set theory to find the candidate triple
    # generate all possible sets and then take away the current sets

    for k,v in scount.items():
        if (v / (len(xextractedtable) - 1)) > 0.5:
            spos = generateAllTuples(k[0], xextractedtable)
            opos = getElementsPosition(scountset_o[k])
            allo = set()
            for x in opos:
                allo = allo.union(generateAllTuples(x, xextractedtable))

            # generate cartesian product between spos and allo and insert predicate into the middle
            combos = list(itertools.product(spos, allo))

            #print(combos)

            # add to list of candidate triples
            for c in combos:
                if c[0][0] == c[1][0] or c[0][0] == None or c[1][0] == None:
                    candidate_triples.add((c[0], PROPERTY_STRING + k[1], c[1]))
                    candidate_preds.add(PROPERTY_STRING + k[1])


    # this might do the same thing, but haven't proved it yet
    for k,v in ocount.items():
        if (v / (len(xextractedtable) - 1)) > 0.5:
            opos = generateAllTuples(k[1], xextractedtable)
            spos = getElementsPosition(ocountset_s[k])
            alls = set()

            for x in spos:
                alls = alls.union(generateAllTuples(x, xextractedtable))

            combos = list(itertools.product(alls, opos))

            for c in combos:
                if c[0][0] == c[1][0] or c[0][0] == None or c[1][0] == None:
                    candidate_triples.add((c[0], PROPERTY_STRING + k[0], c[1]))     
                    candidate_preds.add(PROPERTY_STRING + k[0])
                    
    candidate_triples = candidate_triples - set(xextracted)        
    return candidate_triples, candidate_preds

In [14]:
def getFeatures3457(len_cols, temp_table, article, existing_triples, section_entites = None):
#     perm = list(itertools.permutations(list(range(len_cols))))    
    rcols = []
#     for s1 in range(1, len_rows):
#         for p1 in perm:
#             rcols += list(itertools.product(temp_table[s1][p1[0]],temp_table[s1][p1[1]]))
            
#     # cartesian products also for section heading and article
#     rcols += list(itertools.product(list(range(len_cols)), article))
#     if section_entities is not None:
#         rcols += list(itertools.product(list(range(len_cols)), section_entities))
    
    for s1 in range(1, len_rows):
        ft = flatten(temp_table[s1])
        rcols += list(itertools.permutations(ft,2))
        rcols += list(itertools.product(ft, [article]))
        rcols += list(itertools.product([article], ft))
        if section_entities is not None:
            rcols += list(itertools.product(ft, section_entities))
            rcols += list(itertools.product(section_entities, ft))
        

    allsubj_s = set()
    allobj_s = set()
    allsubj = list()
    allobj = list()

    for o in set(rcols):
        allsubj_s.add(o[0])
        allobj_s.add(o[1])
        allsubj.append(o[0])
        allobj.append(o[1])
        
    # get the (s,o) from temp_table and existing_triples
    etriples = set()
    for x in existing_triples:
        etriples.add((getValueFromTable(x[0], temp_table,article,section_entities),\
                        getValueFromTable(x[2], temp_table,article,section_entities)))
    
    print(etriples)

    feature3 = len(rcols)
    feature4 = len(rcols) - len(etriples)
    feature5 = len(set(rcols)) - len(etriples)
    feature7 = (len(allsubj_s)/len(allsubj))/(len(allobj_s)/len(allobj))
    
    return feature3, feature4, feature5, feature7, rcols

In [15]:
tables = nt1_2['Db4o']

k = 'Db4o'

# for k,v in tables.items():
#     print(k)
#     for k1,v1 in v.items():
for v in range(1):
    print(k)
    for k1,v1 in tables.items():
        # hit table
        old_table = v1["old_table"]
        new_table = v1["new_table"]
        section_title = v1["section_title"]

        len_rows = len(old_table)
        len_cols = len(old_table[0])

        temp_table = [[0 for y in range(len_cols)] for x in range(len_rows)]

    #     # look at relationships between header (predicate) and cell (subject or object)
    #     # header case
    #     # disambiguate header (get first 3 results from wikidata search of header string)
    #     header_pred = dict()
    #     for y in range(len_cols):
    #         header_pred[str(y)] = findProperty(old_table[0][y], 3)

        # populate temp table (copy of new table) with values from original table if it hasn't been disambiguated
        existing_triples = list() # contains list of (pos,predicate, pos)
        predicates = set()
        for x in range(len_rows):
            for y in range(len_cols):
                if not new_table[x][y]:
                    # try to make the obj a number
                    n = find_number(old_table[x][y])
                    if n:
                        temp_table[x][y] = [str(n[0])]
                    else:
                        temp_table[x][y] = [str(old_table[x][y]).replace("'",'"')]
                else:
                    temp_table[x][y] = new_table[x][y]


            fnt = [x1 for x1 in flatten(new_table[x]) if x1!='']
            if x > 0:
                pairs = get_pairs([x1 for x1 in flatten(temp_table[x]) if x1!=''])
                for p in pairs:
                    predicates = predicates.union(getSOPred(p[0], p[1], fnt, existing_triples, temp_table[x]))

        # look at relationships between article title and cells
        # look at relationships betweeen section title (if able to be disambiguated) and cells

        # first disambiguate section title by combining article title, summary and section title
        section_entities = list()
        d = disambig(k + retrieveExtract(k) + section_title, 0.1)
        for original,entitytitle in d.items():
            if original in section_title:
                section_entities.append(entitytitle)    

        # skip the header row
        for x in range(1,len_rows):
            cells = [x1 for x1 in flatten(temp_table[x]) if x1!='']
            fnt = [x1 for x1 in flatten(new_table[x]) if x1!='']        
            for c in cells:
                # match each cell with the header and possibly subsection title
                if section_entities:
                    for ise, se in enumerate(section_entities):
                        predicates = predicates.union(getSOPred(se, c, fnt, existing_triples, temp_table[x], section_entities, (None,(ise, "sub"))))
                        feature3, feature4, feature5, feature7, allsocombos = getFeatures3457(len_cols, temp_table, k, existing_triples, section_entities)
                else:
                    feature3, feature4, feature5, feature7, allsocombos = getFeatures3457(len_cols, temp_table, k, existing_triples)

                predicates = predicates.union(getSOPred(k, c, fnt, existing_triples, row = temp_table[x], pos = (None, (None, "article"))))
                
  
        # based on exisiting triples, suggest candidate triples
        # if the existing predicate happens > 50% in the same indices we suggest it
#         xextractedtable = temp_table
#         xextracted = existing_triples
        feature1 = len(temp_table) - 1
        feature2 = len(temp_table[0])
        # retrive feature 6 later
        # features 8-12 are calculated later
        # feature 13,14,15
        
        
        uniquepotential, genpreds = generateCandidates(existing_triples,temp_table)

        
        

Db4o
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)'), ('Software versioning', 'Software release life cycle')}
{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)'), ('Software versioning', 'Software release life cycle')}
{('Db4o', 

In [18]:
getFeatures3457(len_cols, temp_table, k, existing_triples, section_entities)

{('Db4o', 'Java (programming language)'), ('Eclipse (software)', 'Java (programming language)'), ('Software versioning', 'Software release life cycle')}


(378,
 375,
 202,
 1.0,
 [('Db4o', 'Software developer'),
  ('Db4o', 'CONFIG.SYS'),
  ('Db4o', 'ASP.NET'),
  ('Db4o', 'COM file'),
  ('Db4o', 'Directory (computing)'),
  ('Db4o', 'Hypertext Transfer Protocol'),
  ('Db4o', 'Software release life cycle'),
  ('Db4o', 'Office Open XML'),
  ('Db4o', 'Java (programming language)'),
  ('Software developer', 'Db4o'),
  ('Software developer', 'CONFIG.SYS'),
  ('Software developer', 'ASP.NET'),
  ('Software developer', 'COM file'),
  ('Software developer', 'Directory (computing)'),
  ('Software developer', 'Hypertext Transfer Protocol'),
  ('Software developer', 'Software release life cycle'),
  ('Software developer', 'Office Open XML'),
  ('Software developer', 'Java (programming language)'),
  ('CONFIG.SYS', 'Db4o'),
  ('CONFIG.SYS', 'Software developer'),
  ('CONFIG.SYS', 'ASP.NET'),
  ('CONFIG.SYS', 'COM file'),
  ('CONFIG.SYS', 'Directory (computing)'),
  ('CONFIG.SYS', 'Hypertext Transfer Protocol'),
  ('CONFIG.SYS', 'Software release life

In [ ]:
existing_triples

In [ ]:
import csv

# now that table has been recreated with only disambiguated entities
# let the magic happen
# extract all the triples and features

def addTripleCSV(d, file, mode):
    with open(file, mode, newline='') as csvfile:
        fieldnames = ['id', 'subject', 'predicate','object']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if mode == 'w':
            writer.writeheader()

        writer.writerow(d)